# SurvivalLCS Experiment Runs

## Import and Setup

### Load packages

In [190]:
import os
import pandas as pd
import numpy as np
import random
import sys
import glob
from random import shuffle
from random import sample
import matplotlib.pyplot as plt
import sys
import shutil
import pickle
from survival_ExSTraCS import ExSTraCS

In [191]:
sys.path.append("/home/bandheyh/common/survival-lcs")

In [192]:
plt.ioff()
plt.ioff()

## Survival-LCS Parameters

### Set file names and necessary parameters

In [193]:
# parameter to run using hpc resources
HPC = True

homedir = "/home/bandheyh/common/survival-lcs/pipeline_copy"
models = ['me', 'epi', 'het', 'add']
m0s = []

c = [0.1,0.4,0.8]
nfeat = ['f100'] #add f10000 when on cluster
maf = ['maf0.2','maf0.4']

iterations = 50000
cv_splits = 5

DEBUG = False
if DEBUG:
    models = ['me']
    c = [0.1]
    nfeat = ['f100']
    maf = ['maf0.2', 'maf0.4']
    iterations = 1000
    cv_splits = 3

### Create empty brier score DataFrame
brier_df = pd.DataFrame()
cox_brier_df = pd.DataFrame()

# other non-parameters

simulated = True # CHANGE THIS TO FALSE IF RUNNING REAL DATA

lcs_run = True
dtype_list = []

### Run the survival_LCS pipeline

In [194]:
def get_parameters(models, nfeat, maf, i, j, k):

    g = homedir + '/' + 'simulated_datasets/' + \
        'EDM-1_one_of_each/'+str(models[i]) + \
        '_' + str(nfeat[j]) + '_' + str(maf[k]) + '_' + 'EDM-1_01.txt'
    dtype = str(models[i]) + '_' + str(nfeat[j]) + '_' + str(maf[k])
    dtype_list.append(dtype)
    print(g)

    d = homedir + '/' + 'cv_sim_data/cv_' + str(models[i]) + '/' + dtype
    m = homedir + '/' + 'pickled_cv_models/' + str(models[i]) + '/' + dtype
    o = homedir + '/' + 'sim_lcs_output/' + str(models[i]) + '/' + dtype

    ### Set m0_path
    if models[i] in ['me','add','het']:
        m0_path = homedir+'/'+'simulated_datasets/'+'EDM-1_one_of_each/model_files/me_h0.2_'+str(maf[k])+'_Models.txt'
    else:
        m0_path = homedir+'/'+'simulated_datasets/'+'EDM-1_one_of_each/model_files/epi_h0.2_'+str(maf[k])+'_Models.txt'

    ### Set m1_path
    if models[i] in ['me','epi']:
        m1_path = None
    else:
        m1_path = homedir+'/'+'simulated_datasets/'+'EDM-1_one_of_each/model_files/epi_h0.2_'+str(maf[k])+'_Models.txt'

    ### Set m0_type
    if models[i] in ['me','add','het']:
        m0_type = 'main_effect'
    else:
        m0_type = '2way_epistasis'

    ### Set m1_type
    if models[i] in ['me', 'epi']:
        m1_type = None
    else:
        m1_type = '2way_epistasis'

    ### Set mtype
    if models[i] == 'me':
        mtype = 'main_effect'
    elif models[i] == 'epi':
        mtype = '2way_epistasis'
    elif models[i] == 'add':
        mtype = 'additive'
    else:
        mtype = 'heterogeneous'


    e = "testallsims"
    print(str(models[i])+'_'+str(nfeat[j])+'_'+str(maf[k]))

    return g, mtype, d, m, o, e,brier_df,cox_brier_df, m0_path, m0_type, m1_path, m1_type



In [195]:
from survival_ExSTraCS import ExSTraCS
class ExSTraCSWraper:
    def __init__(self, g, mtype, d, m, o, e, brier_df, cox_brier_df, 
                 m0_path = None, m0_type = None, m1_path = None, m1_type = None,
                 c = 0.1, cv=0, perm_n = 0, T = 100, k = 8, 
                 time_label = "eventTime", status_label = "eventStatus", instance_label="inst", 
                 random_state = None, iterations = 50000, 
                 nu = 1, rp = 1000):
        self.gametes_data_path = g
        self.gametes_model_path_0 = m0_path
        self.gametes_model_path_1 = m1_path
        self.data_path = d
        self.model_path = m
        self.output_path = o
        self.experiment_name = e
        self.model0_type = m0_type
        self.model1_type = m1_type
        self.model_type = mtype #add parameter with name of original dataset


        self.time_label = time_label
        self.status_label = status_label
        self.instance_label = instance_label
        self.T = T
        self.knots = k
        self.censor = c

        self.iterations = iterations
        self.random_state = random_state

        self.cv_count = cv
        self.nu = nu
        self.rulepop = rp

        if self.random_state == None:
            self.random_state = random.randint(0, 1000000)
        else:
            self.random_state = int(self.random_state)
        self.perm_n = perm_n


    def fit(self):
        train_file = self.data_path+ '/' + str(self.model_type) + '_cens'+ \
            str(self.censor) + '_surv_'+ str('2024-04-07') + '_CV_'+str(self.cv_count)+'_Train.txt'
        data_train = pd.read_csv(train_file, sep='\t') #, header = 0
        timeLabel = self.time_label
        censorLabel = self.status_label
        instID = self.instance_label

        # Derive the attribute and phenotype array using the phenotype label
        dataFeatures_train = data_train.drop([timeLabel,censorLabel,instID],axis = 1).values
        dataEvents_train = data_train[[timeLabel,censorLabel]].values

        # split dataEvents into two separate arrays (time and censoring)
        dataEventTimes_train = dataEvents_train[:,0]
        dataEventStatus_train = dataEvents_train[:,1]


        test_file = self.data_path+ '/' + str(self.model_type) + '_cens'+ str(self.censor)\
              + '_surv_'+ str('2024-04-07') + '_CV_'+str(self.cv_count)+'_Test.txt'
        data_test = pd.read_csv(test_file, sep='\t') #, headers = 0
        timeLabel = self.time_label
        censorLabel = self.status_label

        np.random.shuffle(dataEventTimes_train)

        ### Train the survival_ExSTraCS model
        model = ExSTraCS(learning_iterations = self.iterations,nu=self.nu,N=self.rulepop)
        self.trainedModel = model.fit(dataFeatures_train,dataEventTimes_train,dataEventStatus_train)

    def brier_score(self):
        train_file = self.data_path+ '/' + str(self.model_type) + '_cens'+ \
            str(self.censor) + '_surv_'+ str('2024-04-07') + '_CV_'+str(self.cv_count)+'_Train.txt'
        data_train = pd.read_csv(train_file, sep='\t') #, header = 0
        timeLabel = self.time_label
        censorLabel = self.status_label
        instID = self.instance_label

        #Derive the attribute and phenotype array using the phenotype label
        dataFeatures_train = data_train.drop([timeLabel,censorLabel,instID],axis = 1).values
        dataEvents_train = data_train[[timeLabel,censorLabel]].values

        #split dataEvents into two separate arrays (time and censoring)
        dataEventTimes_train = dataEvents_train[:,0]
        dataEventStatus_train = dataEvents_train[:,1]


        test_file = self.data_path+ '/' + str(self.model_type) + '_cens'+ str(self.censor)\
              + '_surv_'+ str('2024-04-07') + '_CV_'+str(self.cv_count)+'_Test.txt'
        data_test = pd.read_csv(test_file, sep='\t') #, headers = 0
        timeLabel = self.time_label
        censorLabel = self.status_label

        #Derive the attribute and phenotype array using the phenotype label
        dataFeatures_test = data_test.drop([timeLabel,censorLabel,instID],axis = 1).values
        dataEvents_test = data_test[[timeLabel,censorLabel]].values

        #split dataEvents into two separate arrays (time and censoring)
        dataEventTimes_test = dataEvents_test[:,0]
        dataEventStatus_test = dataEvents_test[:,1]
        scoreEvents_train = np.flip(dataEvents_train, 1)
        scoreEvents_test = np.flip(dataEvents_test, 1)

        scoreEvents_train = np.core.records.fromarrays(scoreEvents_train.transpose(),names='cens, time', formats = '?, <f8')
        scoreEvents_test = np.core.records.fromarrays(scoreEvents_test.transpose(),names='cens, time', formats = '?, <f8')


        ### Convert float data to int
        dataEventTimes_train = dataEventTimes_train.astype('int64')
        dataEventTimes_test = dataEventTimes_test.astype('int64')
        dataEventStatus_train = dataEventStatus_train.astype('int64')
        dataEventStatus_test = dataEventStatus_test.astype('int64')

        try:
            times, b_scores = self.trainedModel.brier_score(dataFeatures_test, 
                                                            dataEventStatus_test,
                                                            dataEventTimes_test, 
                                                            dataEventTimes_train,
                                                            scoreEvents_train,
                                                            scoreEvents_test)
            tb = pd.DataFrame({'times': times, 'b_scores_' + \
                               str(os.path.basename(self.output_path)) + \
                                '_cens'+ str(self.censor) + \
                                    '_perm' + str(self.perm_n) + \
                                        '_cv' + str(self.cv_count): b_scores})
            tb.to_csv(homedir + '/perm/' + str(os.path.basename(self.output_path)) + \
                                '_cens'+ str(self.censor) + \
                                    '_perm' + str(self.perm_n) + \
                                        '_cv' + str(self.cv_count) + '.csv', index=False)
            tb.set_index('times',inplace=True)
            return tb
        except Exception as e:
            return e

    def return_ibs(self):
        df = self.brier_score(self)
        col_name = 'b_scores_' + \
                               str(os.path.basename(self.output_path)) + \
                                '_cens'+ str(self.censor) + \
                                    '_perm' + str(self.perm_n) + \
                                        '_cv' + str(self.cv_count)
        temp_df = df[[col_name, 'times']]
        temp_df = temp_df.dropna()
        try:
            val = np.trapz(temp_df[col_name], temp_df['times']) / (list(temp_df['times'])[-1] - list(temp_df['times'])[0])
        except Exception as e:
    #         print(col_name, e)
            val = np.nan
        return val

In [196]:
perm_n = 20

In [197]:
error_idxs = [2056]

In [198]:
job_obj_list = list()
arr = np.arange(len(models) * len(nfeat) * len(maf) * len(c) * \
                perm_n * cv_splits).reshape(len(models), len(nfeat), len(maf), len(c), perm_n, cv_splits)

# Convert a 1D index to a 3D index
for x in error_idxs:
    i, j, k, ii, jj, kk = np.unravel_index(x, arr.shape)
    print(models[i], nfeat[j], maf[k], c[ii], jj, kk)
    g, mtype, d, m, o, e,brier_df,cox_brier_df, \
        m0_path, m0_type, m1_path, m1_type = get_parameters(models, nfeat, maf, i, j, k)
    print(ii, jj, kk)
    obj = ExSTraCSWraper(g, mtype, d, m, o, e, brier_df, cox_brier_df, 
                            m0_path, m0_type, m1_path, m1_type,
                            c = c[ii], cv=kk, perm_n = jj)
    job_obj_list.append(obj)

add f100 maf0.2 0.8 11 1
/home/bandheyh/common/survival-lcs/pipeline_copy/simulated_datasets/EDM-1_one_of_each/add_f100_maf0.2_EDM-1_01.txt
add_f100_maf0.2
2 11 1


## HPC Code

In [199]:
import dask
from dask.distributed import Client
from dask_jobqueue import SLURMCluster, LSFCluster, SGECluster

In [200]:
def get_cluster(cluster_type='SLURM', output_path=".", queue='defq', memory=4):
    client = None
    try:
        if cluster_type == 'SLURM':
            cluster = SLURMCluster(queue=queue,
                                   cores=1,
                                   memory=str(memory) + "G",
                                   walltime="24:00:00",
                                   log_directory=output_path + "/dask_logs/")
            cluster.adapt(maximum_jobs=400)
        elif cluster_type == "LSF":
            cluster = LSFCluster(queue=queue,
                                 cores=1,
                                 mem=memory * 1000000000,
                                 memory=str(memory) + "G",
                                 walltime="24:00",
                                 log_directory=output_path + "/dask_logs/")
            cluster.adapt(maximum_jobs=400)
        elif cluster_type == 'UGE':
            cluster = SGECluster(queue=queue,
                                 cores=1,
                                 memory=str(memory) + "G",
                                 resource_spec="mem_free=" + str(memory) + "G",
                                 walltime="24:00:00",
                                 log_directory=output_path + "/dask_logs/")
            cluster.adapt(maximum_jobs=400)
        elif cluster_type == 'Local':
            c = Client()
            cluster = c.cluster
        else:
            raise Exception("Unknown or Unsupported Cluster Type")
        client = Client(cluster)
    except Exception as e:
        print(e)
        raise Exception("Exception: Unknown Exception")
    print("Running dask-cluster")
    print(client.scheduler_info())
    return client

In [201]:
cluster = get_cluster(output_path=homedir)

/home/bandheyh/common/anaconda3/envs/slcs/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37761 instead
  warnings.warn(


Running dask-cluster
{'type': 'Scheduler', 'id': 'Scheduler-aecaa8e5-03f6-45a1-8f13-0364b9ebe653', 'address': 'tcp://10.17.134.112:39525', 'services': {'dashboard': 37761}, 'started': 1712863512.137886, 'workers': {}}


In [202]:
def run_parallel(modelwraper):
    try:
        modelwraper.fit()
        brier_df = modelwraper.brier_score()
    except Exception as e:
        raise e
        brier_df = e
    return brier_df

In [203]:
job_obj_list[0], len(job_obj_list)

(<__main__.ExSTraCSWraper at 0x1555121a2bb0>, 1)

In [204]:
if HPC == True:
    delayed_results = []
    for model in job_obj_list:
        brier_df = dask.delayed(run_parallel)(model)
        delayed_results.append(brier_df)
    results = dask.compute(*delayed_results)

In [205]:
# if HPC:
#     results = dask.compute([dask.delayed(run_parallel)(model) for model in job_obj_list])

In [206]:
# with open(homedir+'/results_perm_parallel_3.pkl', 'wb') as file:
#     pickle.dump(results, file, pickle.HIGHEST_PROTOCOL)

### Error Checking

In [207]:
error_idxs = list()
for i in range(len(results)):
    if type(results[i]) ==  ValueError:
        print(i, results[i])
        error_idxs.append(i)

In [208]:
error_idxs = list()
for i in range(len(results)):
    if type(results[i]) ==  ValueError:
        print(i, results[i])
        error_idxs.append(i)

### IBS Tables

In [209]:
# brier_df_list = list()
# arr = np.arange(len(results)).reshape(len(models), len(nfeat), len(maf), len(c), perm_n, cv_splits)
# for x in range(len(results)):
#     i, j, k, ii, jj, kk = np.unravel_index(x, arr.shape)
#     print(models[i], nfeat[j], maf[k], c[ii], jj, kk)
#     current_ibs = results[x]
#     brier_df_list.append(current_ibs)
# brier_df = pd.concat(brier_df_list, axis = 1, sort = False).reset_index()
# brier_df.to_csv(homedir+'/perm_ibs_data_all_parallel.csv', index = False)
# brier_df

In [210]:
error_idxs_org = [2056]

In [211]:
np.array(error_idxs_org)[error_idxs]

array([], dtype=int64)